In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt

# Set file path and read file
file_path = "medical_clean.csv"
df = pd.read_csv(file_path, index_col=0)

# Display data types
df.info()

# Function to convert columns to bool
def convert_to_boolean(df, columns):
    for col in columns:
        try:
            df[col] = df[col].str.startswith('Yes')
        except AttributeError:
            print(f"{col}'not a string column. Next!")  
        df[col] = df[col].astype(bool)
    return df

# Function to convert category and assign numerics
def convert_to_category(df, column, categories, levels):
    df[column] = df[column].astype("category")
    df[column] = df[column].cat.set_categories(categories).cat.codes
    return df

# Convert columns to bool
boolean_columns = [
    'ReAdmis', 'Soft_drink', 'HighBlood', 'Stroke', 'Overweight', 'Arthritis', 
    'Diabetes', 'Hyperlipidemia','BackPain','Anxiety', 'Allergic_rhinitis', 
    'Reflux_esophagitis', 'Asthma'
]
df = convert_to_boolean(df, boolean_columns)

# Convert category and assign numerics
df = convert_to_category(df, "Marital", ['Divorced', 'Married', 'Never Married', 'Separated', 'Widowed'], range(5))
df = convert_to_category(df, "Gender", ['Female', 'Male', 'Nonbinary'], range(3))
df = convert_to_category(df, "Initial_admin", ['Elective Admission', 'Emergency Admission', 'Observation Admission'], range(3))
df = convert_to_category(df, "Complication_risk", ['High', 'Medium', 'Low'], range(3))
df = convert_to_category(df, "Services",['Blood Work', 'CT Scan', 'Intravenous', 'MRI'], range(4))

# Display dtypes after changes
df.info()

# Create a pivot table
pivot_table = pd.pivot_table(df, index='HighBlood', columns='ReAdmis', aggfunc='size', fill_value=0)
print(pivot_table)

# Perform chi-square test
chi2, p_value, _, _ = chi2_contingency(pivot_table)

# Print the results
print("Chi-square statistic:", chi2)
print("P-value:", p_value)
print(f"The p-value, or probability under the null hypothesis of obtaining a result as extreme as the one observed, is {p_value:.3f}.")


# Univariate categorical variable 1
# Percent by Initial Admission type
plt.subplot(1, 2, 1)
plt.title("Patient Initial Admission")
admin_percent = df["Initial_admin"].value_counts()
admin_labels = [ "Emergency Admission","Elective Admission", "Observation"]
plt.pie(admin_percent, labels=admin_labels, autopct='%1.1f%%', startangle=90, counterclock=False)
plt.axis('square')
plt.show()

# Mapping to numeric value
value_descriptions = {
  
    0: "Elective Admission",
    1: "Emergency Admission",
    2: "Observation"
}
value_counts = df['Initial_admin'].value_counts(normalize=True) * 100
# Get description from dictionary, default to "Unknown" if not found
for value, count in value_counts.items():
    description = value_descriptions.get(value, "Unknown")  
    print(f"{description}: {count:.2f}%")
#End Univariate categorical variable 1
    
    
# Univariate categorical variable 2
# Percent of patients recieving each service
service_percent = df["Services"].value_counts()
sample_size = len(df)
service_percentages = (service_percent / sample_size) * 100
custom_labels = {0: "Blood Work", 1: "Intravenous", 2: "CT Scan", 3: "MRI"}

plt.figure(figsize=[10, 6])
plt.bar(service_percentages.index, service_percentages, color='skyblue')
plt.title("Primary Service Received While Hospitalized")
plt.xlabel("Service")
plt.ylabel("Percentage")
plt.xticks(rotation=45, ticks=service_percentages.index, labels=[custom_labels.get(i, '') for i in service_percentages.index])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Mapping to numeric values
value_descriptions = {
    0: "Blood Work",
    1: "Intravenous",
    2: "CT Scan",
    3: "MRI",
}
# Info on services
value_counts = df['Services'].value_counts(normalize=True) * 100
for value, count in value_counts.items():
    description = value_descriptions.get(value, "Unknown")  
    print(f"{description}: {count:.2f}%")
#End Univariate categorical variable 2
    
# Univariate continuous varible 1
# Initial days in hospital
plt.figure(figsize=[8, 6])
sns.violinplot(data=df, x="Initial_days", color='skyblue') 
plt.title("Distribution of Initial Days Patient is in Hospital")
plt.xlabel("Initial Days in Hospital") 
plt.grid(axis='x', linestyle='--', alpha=0.7)  
plt.show()

# Measures of Central Tendency	
print("Summary statistics of 'Initial Days':")
print(df['Initial_days'].describe())
#End Univariate continuous varible 1

# Univariate continuous varible 2
# Total Charges by number of patients
plt.figure(figsize=[8, 6])
plt.hist(df["TotalCharge"], bins=30, color='skyblue', edgecolor='black')
plt.title("Distribution of Total Charges(Avg per patient by # of days)")
plt.xlabel("Total Charges")
plt.ylabel("Number of Patients")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Measures of Central Tendency	
print("Summary statistics of 'Total Charges':")
print(df['TotalCharge'].describe())
#End Univariate continuous varible 2






# Bivariate Statistic 1
# Complication Risk info
print("Unique values of 'Complication_risk' and their counts:")
print(df['Complication_risk'].value_counts())
# Mapping for Complication Risk categories
risk_mapping = {0: 'Low', 1: 'Medium', 2: 'High'}
df['Complication_risk'] = df['Complication_risk'].map(risk_mapping)
# Additional Charges info
print("Summary statistics of 'Additional_charges':")
print(df['Additional_charges'].describe())
# Create a violin plot for Additional Charges and Complication Risk
plt.figure(figsize=(10, 6))
sns.violinplot(data=df, x='Complication_risk', y='Additional_charges', order=['Low', 'Medium', 'High'], palette='viridis')
plt.xlabel('Complication Risk')
plt.ylabel('Additional Charges')
plt.title('Complication Risk vs Additional Charges')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
#End Bivariate Statistic 1

# Bivariate 2
# Measures of Central Tendency	
print("Summary statistics of 'Total Charges':")
print(df['TotalCharge'].describe())

# Gender info
print("Unique values of 'Gender' and their counts:")
print(df['Gender'].value_counts())


# Create a box plot for Total Charges and Gender
plt.figure(figsize=(8, 6))
sns.boxplot(data=df, x='Gender', y='TotalCharge', palette='muted')
plt.xlabel('Gender')
plt.ylabel('Total Charges')
plt.title('Total Charges by Gender')
plt.grid(axis='y', linestyle='--', alpha=0.7)
legend_labels = ['Female(0)', 'Male(1)', 'Nonbinary(2)']
colors = sns.color_palette('muted')[:3]
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=color, markersize=10, label=label)
                  for color, label in zip(colors, legend_labels)]
plt.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(1, 1))
plt.show()
# End Bivariate 2